<a href="https://colab.research.google.com/github/julius-banda/OTP/blob/master/MyCracklateet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install scikit-learn
!pip install matplotlib
!pip install tensorflow
!pip install scikit-learn
!pip install matplotlib


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import random
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, LeakyReLU, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Configuration
CONFIG = {
    "data_file": "/content/drive/MyDrive/rounds_data.json",
    "model_file": "/content/drive/MyDrive/round_predictor_model.keras",
    "min_rounds": 50,
    "epochs": 200,
    "batch_size": 16,
    "learning_rate": 0.001,
    "patience": 10,
    "k_folds": 5,
    "noise_factor": 0.05,
    "scale_range": (0.8, 1.2),
}

# Utility Functions
def save_data(rounds_data):
    """Save rounds data to the configured file."""
    with open(CONFIG["data_file"], "w") as f:
        json.dump(rounds_data, f)

def load_data():
    """Load rounds data from the configured file, or return an empty list if not found."""
    if os.path.exists(CONFIG["data_file"]):
        with open(CONFIG["data_file"], "r") as f:
            return json.load(f)
    return []

def classify_color(result):
    """Classify the result into a color category."""
    if 1.00 <= result <= 1.99:
        return "BLUE"
    elif 2.00 <= result <= 9.99:
        return "PURPLE"
    elif result >= 10.00:
        return "PINK"
    return "UNKNOWN"

def determine_accuracy_score(actual, predicted):
    """Determine the accuracy score based on the error between actual and predicted values."""
    error = abs(actual - predicted)
    if error < 1.0:
        return 10  # Perfect prediction
    elif error < 2.0:
        return 9
    elif error < 3.0:
        return 8
    elif error < 5.0:
        return 7
    elif error < 7.0:
        return 6
    elif error < 9.0:
        return 5
    elif error < 11.0:
        return 4
    elif error < 15.0:
        return 3
    elif error < 20.0:
        return 2
    else:
        return 1  # Very inaccurate

def plot_color_patterns(rounds_data):
    """Plot color patterns from the rounds data."""
    colors = {"BLUE": [], "PURPLE": [], "PINK": []}
    for round_data in rounds_data:
        color = round_data["color"]
        if color in colors:
            colors[color].append(round_data["result"])
    for color, results in colors.items():
        plt.scatter(range(len(results)), results, label=color)
    plt.xlabel("Index")
    plt.ylabel("Result")
    plt.legend()
    plt.show()

# Data Preparation Functions
def add_noise_to_data(X, noise_factor):
    """Add random noise to the dataset for augmentation."""
    noise = np.random.normal(0, noise_factor, X.shape)
    return X + noise

def random_scale_features(X, scale_range):
    """Scale features randomly within the given range."""
    scale_factor = np.random.uniform(scale_range[0], scale_range[1], size=(X.shape[1],))
    return X * scale_factor

def detect_outliers(data):
    """Detect outliers using the interquartile range (IQR) method."""
    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return (data < lower_bound) | (data > upper_bound)

def prepare_training_data(rounds_data, augment_data=False):
    """Prepare training data with optional augmentation."""
    X = []
    y = []
    for round_data in rounds_data:
        features = [round_data["round"]]
        color = round_data.get("color")
        if color == "BLUE":
            features.append(0)
        elif color == "PURPLE":
            features.append(1)
        elif color == "PINK":
            features.append(2)
        else:
            features.append(-1)
        X.append(features)
        y.append(round_data["result"])

    X = np.array(X)
    y = np.array(y)

    if augment_data:
        X = add_noise_to_data(X, CONFIG["noise_factor"])
        X = random_scale_features(X, CONFIG["scale_range"])

    outliers = detect_outliers(y)
    X = X[~outliers]
    y = y[~outliers]

    # Normalize targets (y)
    scaler_y = MinMaxScaler()
    y = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

    return X, y, scaler_y

# Feature Engineering
def compute_rolling_averages(data, window=3):
    """Compute rolling averages for the given data."""
    return np.convolve(data, np.ones(window)/window, mode='valid')

def compute_relative_changes(data):
    """Compute relative changes between consecutive data points."""
    return np.diff(data) / data[:-1]

# Model Building
def create_nn_model(input_shape):
    """Create a neural network model with the given input shape."""
    model = Sequential([
        Dense(256, input_dim=input_shape, activation=None),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),
        Dense(128, activation=None),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),
        Dropout(0.4),
        Dense(64, activation=None),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),
        Dropout(0.4),
        Dense(32, activation="relu"),
        Dense(1, activation="linear")
    ])
    model.compile(optimizer=Adam(learning_rate=CONFIG["learning_rate"]), loss="mse", metrics=["mae"])
    return model

# Main Function
def main():
    """Main execution function for training and prediction."""
    rounds_data = load_data()

    if not rounds_data:
        print("No previous data found. Please start entering results of initial rounds.")
        for i in range(CONFIG["min_rounds"]):
            while True:
                try:
                    print(f"Entering data for round {i + 1}...")
                    result = float(input("Enter the actual result: "))
                    break
                except ValueError:
                    print("Invalid input. Please enter a numeric value.")

            color = classify_color(result)
            rounds_data.append({"round": i + 1, "result": result, "color": color})
        save_data(rounds_data)

    if len(rounds_data) < CONFIG["min_rounds"]:
        raise ValueError(f"A minimum of {CONFIG['min_rounds']} rounds is required for prediction.")

    X, y, scaler_y = prepare_training_data(rounds_data, augment_data=True)
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)

    if os.path.exists(CONFIG["model_file"]):
        print("Loading pre-trained model...")
        model = tf.keras.models.load_model(CONFIG["model_file"])
    else:
        print("No pre-trained model found. Creating a new model...")
        model = create_nn_model(input_shape=X.shape[1])

    if not os.path.exists(CONFIG["model_file"]):
        kf = KFold(n_splits=CONFIG["k_folds"], shuffle=True, random_state=42)
        val_scores = []

        for train_index, val_index in kf.split(X_scaled):
            X_train, X_val = X_scaled[train_index], X_scaled[val_index]
            y_train, y_val = y[train_index], y[val_index]

            early_stopping = EarlyStopping(monitor='val_loss', patience=CONFIG["patience"], restore_best_weights=True)
            reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)

            model.fit(X_train, y_train, epochs=CONFIG["epochs"], batch_size=CONFIG["batch_size"],
                      validation_data=(X_val, y_val), callbacks=[early_stopping, reduce_lr], verbose=1)

            val_loss, val_mae = model.evaluate(X_val, y_val, verbose=1)
            val_scores.append((val_loss, val_mae))

        avg_val_loss = np.mean([score[0] for score in val_scores])
        avg_val_mae = np.mean([score[1] for score in val_scores])

        print(f"Average Validation Loss: {avg_val_loss:.4f}")
        print(f"Average Validation MAE: {avg_val_mae:.4f}")

        model.fit(X_scaled, y, epochs=CONFIG["epochs"], batch_size=CONFIG["batch_size"],
                  callbacks=[early_stopping, reduce_lr], verbose=1)

    print("Starting prediction...")
    for i in range(len(rounds_data) + 1, len(rounds_data) + 11):
        features = np.array([[i, -1]])
        features_scaled = scaler.transform(features)
        prediction = model.predict(features_scaled, verbose=0)[0][0]
        prediction_original = scaler_y.inverse_transform([[prediction]])[0][0]
        predicted_color = classify_color(prediction_original)
        print(f"Predicted result for round {i}: {prediction_original:.2f} ({predicted_color})")

        while True:
            try:
                actual_result = float(input(f"Enter the actual result for round {i}: "))
                break
            except ValueError:
                print("Invalid input. Please enter a numeric value.")

        actual_color = classify_color(actual_result)
        accuracy_score = determine_accuracy_score(actual_result, prediction_original)

        print(f"Actual result for round {i}: {actual_result:.2f} ({actual_color})")
        print(f"Accuracy score: {accuracy_score}/10")

        rounds_data.append({
            "round": i,
            "result": actual_result,
            "color": actual_color
        })

    save_data(rounds_data)

    print("Saving the trained model...")
    model.save(CONFIG["model_file"])

if __name__ == "__main__":
    main()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading pre-trained model...
Starting prediction...
Predicted result for round 111: 7.94 (PURPLE)
